In [ ]:
# Install required libraries if not already installed
# !pip install yfinance bs4 lxml pandas matplotlib


In [3]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt


In [4]:
def make_graph(stock_data, revenue_data, stock, revenue, title):
    fig, ax1 = plt.subplots(figsize=(14, 6))

    color = 'tab:red'
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Stock Price', color=color)
    ax1.plot(stock_data.Date, stock_data.Close, color=color, label='Stock Price')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.legend(loc='upper left')

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('Revenue', color=color)
    ax2.plot(revenue_data.Date, revenue_data.Revenue, color=color, label='Revenue')
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.legend(loc='upper right')

    plt.title(title)
    plt.show()


In [5]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [6]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {"User-Agent": "Mozilla/5.0"}
html_data = requests.get(url, headers=headers).text

soup = BeautifulSoup(html_data, "html.parser")
tables = soup.find_all("table")
print("Number of tables found for Tesla:", len(tables))

if len(tables) >= 2:
    target_table = tables[1]
elif len(tables) == 1:
    target_table = tables[0]
else:
    raise Exception("No tables found on Tesla revenue page.")

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in target_table.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$", "").replace(",", "")
        if revenue != '':
            tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame([{"Date": date, "Revenue": revenue}])], ignore_index=True)

tesla_revenue.tail()


Number of tables found for Tesla: 6


,Date,Revenue
58,2010-09-30,31
59,2010-06-30,28
60,2010-03-31,21
61,2009-09-30,46
62,2009-06-30,27


In [19]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 1. Define the URL for GameStop's revenue on Macrotrends
url_gme_revenue = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

# 2. Download the webpage and save the text
try:
    html_data_gme = requests.get(url_gme_revenue, headers=headers, timeout=15).text
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from {url_gme_revenue}: {e}")
    gme_revenue = pd.DataFrame(columns=["Date", "Revenue"]) # Initialize empty DataFrame on error
    print("Initializing empty gme_revenue DataFrame due to request error. No data will be plotted.")
    # Exit here if data fetching is critical, or allow to proceed with empty DF
    # return gme_revenue # If this were a function

# 3. Parse the HTML data using BeautifulSoup
# IMPORTANT: Ensure 'html5lib' is installed in your Python environment.
# If you get 'FeatureNotFound', you need to install it (e.g., pip install html5lib in your terminal).
soup_gme = BeautifulSoup(html_data_gme, "html5lib")


# Initialize an empty DataFrame to store the revenue data
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# --- BEGIN: Table Extraction Logic ---

# Strategy 1: Try pd.read_html with a 'match' argument (most robust if text is consistent)
try:
    gme_revenue_tables = pd.read_html(html_data_gme, match="GameStop Quarterly Revenue", flavor='bs4')
    if gme_revenue_tables:
        gme_revenue = gme_revenue_tables[0] # Take the first table that matches

        # Standardize column names
        if 'GameStop Quarterly Revenue(Millions of US $)' in gme_revenue.columns and \
           'GameStop Quarterly Revenue(Millions of US $).1' in gme_revenue.columns:
            gme_revenue.rename(columns={
                "GameStop Quarterly Revenue(Millions of US $)": "Date",
                "GameStop Quarterly Revenue(Millions of US $).1": "Revenue"
            }, inplace=True)
        elif len(gme_revenue.columns) == 2: # Generic fallback for two columns
            gme_revenue.columns = ["Date", "Revenue"]
        else:
            print("Warning: pd.read_html found table but unexpected column names. Check output.")

    else:
        print("pd.read_html with 'match' found no tables matching 'GameStop Quarterly Revenue'. Trying manual search.")
        raise IndexError # Force fallback to BeautifulSoup manual search

except IndexError:
    # Strategy 2: Manual BeautifulSoup parsing (if pd.read_html fails)
    # This is where you would integrate findings from your manual HTML inspection.
    # EXAMPLE: If you found the table has an ID like <table id="revenue_table">
    # target_table = soup_gme.find('table', id='revenue_table')
    # If no specific ID/class, try finding by specific text within the table:

    target_table = None
    tables = soup_gme.find_all('table')
    for table in tables:
        # Look for specific text that identifies the revenue table
        if "GameStop Quarterly Revenue" in table.get_text():
            target_table = table
            break # Found the table, exit loop

    if target_table:
        print("Found table using BeautifulSoup manual search.")
        for row in target_table.find_all("tr"):
            cols = row.find_all("td")
            if len(cols) >= 2: # Ensure at least two columns (Date, Revenue)
                date_val = cols[0].text.strip()
                revenue_val = cols[1].text.strip().replace("$", "").replace(",", "")
                if revenue_val: # Only add if revenue value is not empty
                    gme_revenue = pd.concat([gme_revenue, pd.DataFrame([{"Date": date_val, "Revenue": revenue_val}])], ignore_index=True)
    else:
        print("Error: Could not find the GameStop Quarterly Revenue table using any method.")
        # If table is not found, gme_revenue will remain empty or as initialized.

# --- END: Table Extraction Logic ---


# 5. Clean and convert 'Revenue' to numeric format
if not gme_revenue.empty: # Only process if data was actually extracted
    gme_revenue["Revenue"] = gme_revenue["Revenue"].astype(str).str.replace(",", "").str.replace("$", "")
    gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"], errors='coerce')


    # 6. Remove any rows where 'Revenue' is NaN (after conversion)
    gme_revenue.dropna(subset=['Revenue'], inplace=True)


    # 7. Convert 'Date' column to datetime objects
    gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'], errors='coerce')
    gme_revenue.dropna(subset=['Date'], inplace=True) # Remove rows with invalid dates


    # 8. Sort the DataFrame by Date in ascending order
    gme_revenue.sort_values(by='Date', inplace=True)
    gme_revenue.reset_index(drop=True, inplace=True) # Reset the index after sorting


# 9. Display the last five rows of the dataframe
print("\nGameStop Revenue Data (last 5 rows):")
print(gme_revenue.tail())

C:\Users\Pooto\AppData\Local\Temp\ipykernel_5284\4110009215.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  gme_revenue_tables = pd.read_html(html_data_gme, match="GameStop Quarterly Revenue", flavor='bs4')



GameStop Revenue Data (last 5 rows):
         Date  Revenue
61 2024-04-30      882
62 2024-07-31      798
63 2024-10-31      860
64 2025-01-31     1283
65 2025-04-30      732


In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd # Ensure pandas is imported for datetime conversion

def make_graph(stock_data, revenue_data, stock_name):
    """
    Generates a dashboard comparing stock price and revenue using Plotly.

    Args:
        stock_data (pd.DataFrame): DataFrame containing stock data with 'Date' and 'Close' columns.
        revenue_data (pd.DataFrame): DataFrame containing revenue data with 'Date' and 'Revenue' columns.
        stock_name (str): Name of the stock for the plot title (e.g., 'Tesla').
    """
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=(f"{stock_name} Historical Share Price", f"{stock_name} Historical Revenue"),
                        vertical_spacing=.3)

    # Ensure Date columns are in datetime format for proper plotting
    # (This step is crucial and was already in previous correct answers)
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'])
    
    # Sort data by Date to ensure correct plotting order
    stock_data = stock_data.sort_values(by='Date')
    revenue_data = revenue_data.sort_values(by='Date')


    # Add Stock Price trace to the first subplot
    fig.add_trace(go.Scatter(x=stock_data.Date, y=stock_data.Close.astype("float"),
                             name="Share Price", mode='lines'), row=1, col=1)

    # Add Revenue trace to the second subplot
    fig.add_trace(go.Scatter(x=revenue_data.Date, y=revenue_data.Revenue.astype("float"),
                             name="Revenue", mode='lines'), row=2, col=1)

    # Update X and Y axes labels
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    # Update layout for overall plot
    fig.update_layout(showlegend=False, height=800,
                      title_text=f"{stock_name} Stock Price and Revenue Analysis",
                      xaxis_rangeslider_visible=True) # This adds the interactive date range slider

    fig.show()

In [23]:
# Assuming 'tesla_data' and 'tesla_revenue' DataFrames are already prepared from previous steps.
# If not, please run the code for Questions 1 and 2 first.

print("\nGenerating Tesla Stock and Revenue Dashboard...")
make_graph(tesla_data, tesla_revenue, 'Tesla (TSLA)')



Generating Tesla Stock and Revenue Dashboard...


In [27]:
# Make sure you have run the code for Question 3 and Question 4 above
# to ensure 'gme_data' and 'gme_revenue' DataFrames are defined.

print("\nGenerating GameStop Stock and Revenue Dashboard...")
make_graph(gme_data, gme_revenue, 'GameStop (GME)')


Generating GameStop Stock and Revenue Dashboard...
